In [12]:
# improts 
import pandas as pd
import ete3
import glob

## Build treee from kraken reports

In [4]:
# do a glob across the kraken files to get all of the files needed 

In [5]:
import glob
from ete3 import NCBITaxa

# Initialize NCBITaxa
ncbi = NCBITaxa()


def parse_all_taxids_from_report(file_path):
    taxids = set()
    with open(file_path, 'r') as file:
        for line in file:
            fields = line.strip().split("\t")
            if len(fields) > 4:
                try:
                    taxid = float(fields[4])
                    if taxid != 0:
                        taxids.add(taxid)
                except ValueError:
                    continue  # Skip malformed lines
    return taxids

# Function to filter TaxIDs that are valid in ETE3
def filter_valid_taxids(taxids):
    valid = []
    for taxid in taxids:
        try:
            ncbi.get_lineage(taxid)
            valid.append(taxid)
        except:
            print(f"Skipping unknown TaxID: {taxid}")
    return valid

# Build a full tree including intermediate nodes
def build_taxonomic_tree(taxids):
    return ncbi.get_topology(taxids, intermediate_nodes=True)

# Save Newick tree
def save_tree(tree, output_file):
    tree.write(format=1, outfile=output_file)
    print(f"Tree written to {output_file}")

In [33]:
# Function to build tree and capture taxid translations
def build_taxonomic_tree_with_mapping(taxids):
    """Build tree and capture all taxid translations"""
    translation_map = {}
    final_taxids = set()
    failed_taxids = []
    
    print("Checking for taxid translations...")
    for taxid in taxids:
        # Skip invalid taxids (0, negative, or non-numeric)
        try:
            taxid_int = int(taxid)
            if taxid_int <= 0:
                failed_taxids.append(taxid)
                continue
        except (ValueError, TypeError):
            failed_taxids.append(taxid)
            continue
            
        try:
            # Get the lineage - this will use the current valid taxid
            lineage = ncbi.get_lineage(taxid_int)
            if lineage:
                # The actual taxid used might be different due to NCBI merging
                current_taxid = lineage[-1]  # Last in lineage is the species
                if current_taxid != taxid_int:
                    translation_map[taxid_int] = current_taxid
                    print(f"Taxid {taxid_int} -> {current_taxid}")
                final_taxids.add(current_taxid)
            else:
                failed_taxids.append(taxid)
                print(f"No lineage found for taxid: {taxid}")
        except Exception as e:
            failed_taxids.append(taxid)
            print(f"Failed to process taxid {taxid}: {str(e)}")
    
    print(f"Successfully processed: {len(final_taxids)} taxids")
    print(f"Failed taxids: {len(failed_taxids)}")
    print(f"Found {len(translation_map)} taxid translations")
    
    if len(final_taxids) == 0:
        raise ValueError("No valid taxids found to build tree")
    
    # Build tree with final taxids
    tree = ncbi.get_topology(list(final_taxids), intermediate_nodes=True)
    
    return tree, translation_map, failed_taxids

# Read in the bracken filtered file and build a tree that corresponds to this 
long_term_table = pd.read_csv('../output_files/long_term_migration_bracken_species_confidence_0.1_icamp_formatted.tsv', sep='\t', index_col=0)

# Clean the taxids - remove any invalid ones before processing
print("Original taxids:", len(long_term_table.index))
print("First few taxids:", list(long_term_table.index[:10]))

# Filter out invalid taxids (0, negative, non-numeric) before validation
valid_indices = []
removed_indices = []

for taxid in long_term_table.index:
    try:
        taxid_int = int(float(taxid))  # Handle both int and float strings
        if taxid_int > 0:
            valid_indices.append(taxid)
        else:
            removed_indices.append(taxid)
    except (ValueError, TypeError):
        removed_indices.append(taxid)

print(f"Removed {len(removed_indices)} invalid taxids (zeros, negatives, non-numeric)")
if removed_indices:
    print("Examples of removed taxids:", removed_indices[:10])

# Filter the table to only valid taxids
long_term_table_clean = long_term_table.loc[valid_indices]
print(f"Clean table: {long_term_table_clean.shape[0]} taxa x {long_term_table_clean.shape[1]} samples")

# Validate the remaining taxids 
filtered_long_term_taxids = filter_valid_taxids(long_term_table_clean.index)
print(f"NCBI-validated taxids: {len(filtered_long_term_taxids)}")

# Build tree and capture translations
if filtered_long_term_taxids:
    tree, translation_map, failed_taxids = build_taxonomic_tree_with_mapping(filtered_long_term_taxids)
    
    # Save the tree
    save_tree(tree, "../data/assembly_processes/long_term_migration_bracken_species_confidence_0.1_tree.nwk")
    
    # Update the table with translated taxids
    print(f"Updating table...")
    
    # Start with the clean table (no invalid taxids)
    updated_table = long_term_table_clean.copy()
    
    # Apply translations if any
    if translation_map:
        print(f"Applying {len(translation_map)} taxid translations...")
        
        # Create new index with translations applied
        new_index = []
        for old_taxid in updated_table.index:
            old_taxid_int = int(float(old_taxid))
            if old_taxid_int in translation_map:
                new_taxid = translation_map[old_taxid_int]
                new_index.append(str(new_taxid))
            else:
                new_index.append(str(old_taxid_int))
        
        # Update the index
        updated_table.index = new_index
        
        # Aggregate rows that now have the same taxid (sum their abundances)
        if len(set(new_index)) < len(new_index):
            print("Aggregating duplicate taxids...")
            updated_table = updated_table.groupby(updated_table.index).sum()
    
    # Remove any taxa that failed NCBI validation
    if failed_taxids:
        failed_taxids_str = [str(int(float(x))) for x in failed_taxids if x in [str(int(float(y))) for y in updated_table.index]]
        if failed_taxids_str:
            print(f"Removing {len(failed_taxids_str)} taxa that failed NCBI validation")
            updated_table = updated_table.drop(index=failed_taxids_str, errors='ignore')
    
    # Save the updated table
    output_file = '../output_files/long_term_migration_bracken_species_confidence_0.1_icamp_formatted_updated.tsv'
    updated_table.to_csv(output_file, sep='\t')
    print(f"Updated table saved to: {output_file}")
    
    # Save the translation mapping for reference
    if translation_map:
        translation_df = pd.DataFrame(list(translation_map.items()), columns=['old_taxid', 'new_taxid'])
        translation_file = '../output_files/taxid_translations.csv'
        translation_df.to_csv(translation_file, index=False)
        print(f"Translation map saved to: {translation_file}")
    
    # Save info about removed taxa
    removal_info = {
        'invalid_taxids': removed_indices,
        'failed_ncbi_validation': failed_taxids
    }
    
    with open('../output_files/removed_taxa_info.txt', 'w') as f:
        f.write("REMOVED TAXA INFORMATION\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Invalid taxids removed (zeros, negatives, non-numeric): {len(removed_indices)}\n")
        for taxid in removed_indices[:20]:  # Show first 20
            f.write(f"  {taxid}\n")
        if len(removed_indices) > 20:
            f.write(f"  ... and {len(removed_indices) - 20} more\n")
        
        f.write(f"\nFailed NCBI validation: {len(failed_taxids)}\n")
        for taxid in failed_taxids[:20]:  # Show first 20
            f.write(f"  {taxid}\n")
        if len(failed_taxids) > 20:
            f.write(f"  ... and {len(failed_taxids) - 20} more\n")
    
    # Print summary
    print("\nSummary:")
    print(f"Original table: {long_term_table.shape[0]} taxa x {long_term_table.shape[1]} samples")
    print(f"After removing invalid taxids: {long_term_table_clean.shape[0]} taxa")
    print(f"Final updated table: {updated_table.shape[0]} taxa x {updated_table.shape[1]} samples")
    print(f"Taxa removed (invalid): {len(removed_indices)}")
    print(f"Taxa removed (NCBI validation failed): {len(failed_taxids)}")
    print(f"Taxa translated: {len(translation_map)}")
        
else:
    print("No valid TaxIDs found to build a tree.")

Original taxids: 4935
First few taxids: [573, 1463165, 1134687, 2675726, 2675710, 2675711, 2675713, 2488567, 2742650, 2697371]
Removed 1 invalid taxids (zeros, negatives, non-numeric)
Examples of removed taxids: [0]
Clean table: 4934 taxa x 22 samples


/home/grig0076/miniconda3/envs/notebook_env/lib/python3.9/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 3112561 was translated into 3020826
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/grig0076/miniconda3/envs/notebook_env/lib/python3.9/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 50340 was translated into 53407
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/grig0076/miniconda3/envs/notebook_env/lib/python3.9/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 621376 was translated into 570156
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/grig0076/miniconda3/envs/notebook_env/lib/python3.9/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 29570 was translated into 77097
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/grig

NCBI-validated taxids: 4934
Checking for taxid translations...
Taxid 3112561 -> 3020826
Taxid 50340 -> 53407
Taxid 621376 -> 570156
Taxid 29570 -> 77097
Taxid 3030639 -> 3030638
Taxid 332410 -> 340145
Taxid 2954088 -> 2954101
Taxid 2745498 -> 1114970
Taxid 2049935 -> 492670
Taxid 237610 -> 47885
Taxid 1415630 -> 3240790
Taxid 2859001 -> 2842348
Taxid 3122409 -> 3141581
Successfully processed: 4926 taxids
Failed taxids: 0
Found 13 taxid translations
Tree written to long_term_migration_bracken_species_confidence_0.1_tree.nwk
Updating table...
Applying 13 taxid translations...
Aggregating duplicate taxids...
Updated table saved to: ../output_files/long_term_migration_bracken_species_confidence_0.1_icamp_formatted_updated.tsv
Translation map saved to: ../output_files/taxid_translations.csv

Summary:
Original table: 4935 taxa x 22 samples
After removing invalid taxids: 4934 taxa
Final updated table: 4926 taxa x 22 samples
Taxa removed (invalid): 1
Taxa removed (NCBI validation failed): 0
Ta

In [24]:
# read in the bracken filtered file and build a tree that corresponds to this 
long_term_table = pd.read_csv('../output_files/long_term_migration_bracken_species_confidence_0.1_icamp_formatted.tsv', sep='\t', index_col=0 )
long_term_taxids = long_term_table.index 

# validate the taxids 
filtered_long_term_taxids = filter_valid_taxids(long_term_taxids)

# Build and save tree
if filtered_long_term_taxids:
    tree = build_taxonomic_tree(filtered_taxids)
    save_tree(tree, "../data/assembly_processes/long_term_migration_bracken_species_confidence_0.1_tree.nwk")
else:
    print("No valid TaxIDs found to build a tree.")

/home/grig0076/miniconda3/envs/notebook_env/lib/python3.9/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 3112561 was translated into 3020826
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/grig0076/miniconda3/envs/notebook_env/lib/python3.9/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 50340 was translated into 53407
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/grig0076/miniconda3/envs/notebook_env/lib/python3.9/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 621376 was translated into 570156
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/grig0076/miniconda3/envs/notebook_env/lib/python3.9/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 29570 was translated into 77097
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/grig

Tree written to long_term_migration_bracken_species_confidence_0.1_tree.nwk
